Collect CT data from LITS Challenge and convert to .png

In [ ]:
import os
import numpy as np
import nibabel as nib
import cv2
from multiprocessing import Pool, cpu_count
from google.colab import drive

drive.mount('/content/drive')
!rsync -ah --progress "/content/drive/My Drive/LITS Challenge/Training Batch 2" /content/lits_raw

data_dir = "/content/lits_raw/Training Batch 2"
out_dir = '/content/lits_png'
max_slices = 1000

os.makedirs(out_dir, exist_ok=True)

def normalize_ct(x):
    x = np.clip(x, -200, 300)
    x = (x - x.min()) / (x.max() - x.min() + 1e-8)
    return (x * 255).astype(np.uint8)

def process_volume(file_number):
    seg_path = os.path.join(data_dir, f"segmentation-{file_number}.nii")
    vol_path = os.path.join(data_dir, f"volume-{file_number}.nii")

    if not (os.path.exists(seg_path) and os.path.exists(vol_path)):
        return f"Skipped {file_number}"

    seg_nii = nib.load(seg_path)
    vol_nii = nib.load(vol_path)

    seg_proxy = seg_nii.dataobj
    vol_proxy = vol_nii.dataobj

    depth = seg_nii.shape[2]
    num_slices = min(max_slices, depth)
    slice_indices = np.linspace(0, depth - 1, num_slices, dtype=int)

    img_dir = f"{out_dir}/images/{file_number}"
    mask_dir = f"{out_dir}/masks/{file_number}"
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    for i in slice_indices:
        img = normalize_ct(np.array(vol_proxy[:, :, i], dtype=np.float32))
        mask = np.array(seg_proxy[:, :, i], dtype=np.uint8)

        cv2.imwrite(f"{img_dir}/{i}.png", img)
        cv2.imwrite(f"{mask_dir}/{i}.png", mask)

    return f"Done {file_number}"

# 🔎 Automatically find all volumes
file_numbers = sorted([
    int(f.split('-')[1].split('.')[0])
    for f in os.listdir(data_dir)
    if f.startswith("volume-")
])

print("Found volumes:", len(file_numbers))

# 🚀 Parallel processing
num_workers = max(1, cpu_count() - 1)
print("Using workers:", num_workers)

with Pool(num_workers) as p:
    results = p.map(process_volume, file_numbers)

print("\n".join(results))
print("All volumes processed ✅")

Move Files to /content folder from MyDrive

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

!rsync -ah --progress "/content/drive/MyDrive/lits_png.zip" /content/lits_png.zip

Dataset Class

In [5]:
import os
import cv2
import torch
from torch.utils.data import Dataset, random_split

class LiverCTDataset(Dataset):
    def __init__(self, root_dir, output_size=160):
        """
        Dataset for liver CT segmentation.

        Loads grayscale CT images and binary segmentation masks from a directory
        structure, resizes them to a fixed output size, and returns them as tensors.

        Parameters
        ----------
        root_dir : str
            Path to the root directory containing 'images' and 'masks' folders.
        output_size : int, optional
            Target spatial dimension for resizing images and masks. Default is 160.
        """
        self.img_dir = os.path.join(root_dir, "images")
        self.mask_dir = os.path.join(root_dir, "masks")
        self.output_size = output_size

        # List of (image_path, mask_path) tuples
        self.samples = []

        # Walk through case folders to pair images with masks
        for case in sorted(os.listdir(self.img_dir)):
            case_img_dir = os.path.join(self.img_dir, case)
            case_mask_dir = os.path.join(self.mask_dir, case)

            # Skip non-directory entries
            if not os.path.isdir(case_img_dir):
                continue

            # Match each image file with its corresponding mask
            for file in sorted(os.listdir(case_img_dir)):
                img_path = os.path.join(case_img_dir, file)
                mask_path = os.path.join(case_mask_dir, file)

                # Only include samples where both image and mask exist
                if os.path.exists(mask_path):
                    self.samples.append((img_path, mask_path))

        print("Total slices found:", len(self.samples))

    def __len__(self):
        """Return the total number of samples in the dataset."""
        return len(self.samples)

    def __getitem__(self, idx):
        """
        Load and preprocess a single sample.

        Reads grayscale image and mask from disk, resizes both to
        (output_size, output_size), normalizes the image to [0, 1],
        and binarizes the mask.

        Parameters
        ----------
        idx : int
            Index of the sample to retrieve.

        Returns
        -------
        mask : torch.Tensor
            Binary segmentation mask of shape (1, output_size, output_size).
        img : torch.Tensor
            Normalized CT image of shape (1, output_size, output_size).

        Raises
        ------
        ValueError
            If image or mask file cannot be loaded.
        """
        img_path, mask_path = self.samples[idx]

        # Load as grayscale numpy arrays
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            raise ValueError(f"Failed to load image: {img_path}")
        if mask is None:
            raise ValueError(f"Failed to load mask: {mask_path}")

        # Resize image with bilinear interpolation for smooth scaling
        img = cv2.resize(img, (self.output_size, self.output_size), interpolation=cv2.INTER_LINEAR)
        # Resize mask with nearest-neighbor to preserve binary labels
        mask = cv2.resize(mask, (self.output_size, self.output_size), interpolation=cv2.INTER_NEAREST)

        # Convert to tensors and normalize image to [0, 1]
        img = torch.from_numpy(img).unsqueeze(0).float() / 255.0
        # Binarize mask: any non-zero pixel becomes 1.0
        mask = (torch.from_numpy(mask).unsqueeze(0).float() / 255.0 > 0).float()

        return mask, img

Generator and Discriminator Networks

In [6]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_channels):
        """
        Generator model for.

        Parameters
        ----------
        input_channels: int
        Number of input channels.
        """
        super(Generator, self).__init__()

        # Define layers for the generator
        self.layer1 = self.conv_block(input_channels, 64, 4, 2, 1)
        self.layer2 = self.conv_block(64, 128, 4, 2, 1, batch_norm=True)
        self.layer3 = self.conv_block(128, 256, 4, 2, 1, batch_norm=True)
        self.layer4 = self.conv_block(256, 512, 4, 2, 1, batch_norm=True)
        self.layer5 = self.conv_block(512, 256, 3, 1, 1, batch_norm=True)
        self.layer6 = self.conv_block(256, 128, 3, 1, 1, batch_norm=True)
        self.layer7 = self.conv_block(128, 64, 3, 1, 1, batch_norm=True)
        self.layer8 = self.conv_block(64, 1, 3, 1, 1)

    def conv_block(self, in_channels, out_channels, kernel_size, stride, padding, batch_norm=False):
        """
        Defines a convolutional block consisting of convolutional, activation, and optional batch normalization layers.

        Parameters
        ----------
        in_channels: int
            Number of input channels.
        out_channels: int
            Number of output channels.
        kernel_size: int
            Size of the convolutional kernel.
        stride: int
            Stride value for the convolution operation.
        padding: int
            Padding value for the convolution operation.
        batch_norm: bool, optional
            Whether to include batch normalization layer, default is False.

        Returns
        -------
        layers: nn.Sequential
            Sequential container for the convolutional block layers.
        """
        layers = []
        # Add convolutional layer
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding))

        #  Add ReLU activation function
        layers.append(nn.ReLU(inplace=True))

        # Add batch normalization layer if specified
        if batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))
        return nn.Sequential(*layers)

    # Define the forward pass through the generator
    def forward(self, x):
        """
        Forward pass through the generator network.

        Parameters
        ----------
        x: torch.Tensor
            Input tensor.

        Returns
        -------
        x8: torch.Tensor
            Output tensor after passing through the generator.
        """

        x1 = self.layer1(x)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)
        x5 = self.layer5(x4)
        x5 = nn.functional.interpolate(x5, scale_factor=2, mode='nearest')
        x6 = self.layer6(x5)
        x6 = nn.functional.interpolate(x6, scale_factor=2, mode='nearest')
        x7 = self.layer7(x6)
        x7 = nn.functional.interpolate(x7, scale_factor=2, mode='nearest')
        x8 = self.layer8(x7)
        x8 = nn.functional.interpolate(x8, scale_factor=2, mode='nearest')

        return x8

class Discriminator(nn.Module):
    def __init__(self, input_channels):
        """
        Discriminator model for.

        Parameters
        ----------
        input_channels: int
            Number of input channels.
        """
        super(Discriminator, self).__init__()

        # Define layers for the discriminator
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=input_channels,
                      out_channels=128,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=128,
                      out_channels=256,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=256,
                      out_channels=1,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x, layer_idx=None):
        """
        Forward pass through the discriminator network.

        Parameters
        ----------
        x: torch.Tensor
            Input tensor.
        layer_idx : int or None, optional
            Index of the layer up to which features are extracted.
            If None, returns the output of the discriminator.
            Default is None.

        Returns
        -------
        flattened_features : torch.Tensor
            Output tensor after passing through the discriminator.
            If `layer_idx` is specified, returns flattened features up to the specified layer.
        """
        if layer_idx is None:
            # Normal forward pass through discriminator
            x = self.model(x)
            return x
        else:
            # List to extract features up to the specified layer
            hierarchical_features = []

            for i in range(layer_idx + 1):
                x = self.model[i * 3:i * 3 + 3](x)
                hierarchical_features.append(x)

            # Concatenate the flattened tensors along dim 1 dimension to allow for ease of calculate of the loss
            flattened_features = torch.cat([torch.flatten(f, start_dim=1) for f in hierarchical_features], dim=1)

            return flattened_features

    def forward_all_layers(self, x):
        """Extract features from all layers in a single forward pass."""
        features = []
        for i in range(3):
            x = self.model[i * 3:i * 3 + 3](x)
            features.append(x)
        return features

In [7]:
import torch
import torch.nn as nn

class L1Loss(nn.Module):
    def __init__(self, critic_network, num_layers, num_training_images):
        """
        Multi-layer perceptual L1 loss function.

        Uses the discriminator's hierarchical features to compute L1 distance
        between masked predictions and masked ground truth at multiple network
        depths, encouraging perceptually realistic segmentations.

        Parameters
        ----------
        critic_network : nn.Module
            Discriminator network with a `forward_all_layers` method that
            returns a list of feature maps from each layer.
        num_layers : int
            Number of layers in the discriminator to extract features from.
        num_training_images : int
            Batch size (kept for API compatibility).
        """
        super(L1Loss, self).__init__()
        self.critic_network = critic_network
        self.num_layers = num_layers
        self.num_training_images = num_training_images

    def forward(self, original_images, predicted_labels, ground_truth_labels):
        """
        Compute the multi-layer perceptual L1 loss.

        Masks the original images by both predicted and ground truth labels,
        extracts hierarchical features from the discriminator for each,
        and computes the average L1 distance across all layers.

        Parameters
        ----------
        original_images : torch.Tensor
            Input CT images of shape (B, 1, H, W).
        predicted_labels : torch.Tensor
            Predicted segmentation masks of shape (B, 1, H, W).
        ground_truth_labels : torch.Tensor
            Ground truth segmentation masks of shape (B, 1, H, W).

        Returns
        -------
        average_loss : torch.Tensor
            Mean L1 loss averaged across all discriminator layers.
        """
        # Extract features from all discriminator layers in a single forward pass
        features_predicted = self.critic_network.forward_all_layers(original_images * predicted_labels)
        features_ground_truth = self.critic_network.forward_all_layers(original_images * ground_truth_labels)

        # Accumulate L1 distance across each layer
        total_loss = 0.0
        for feat_pred, feat_gt in zip(features_predicted, features_ground_truth):
            # Flatten spatial dimensions for element-wise comparison
            flat_pred = torch.flatten(feat_pred, start_dim=1)
            flat_gt = torch.flatten(feat_gt, start_dim=1)
            # Mean Absolute Error between feature representations
            total_loss += (flat_pred - flat_gt).abs().mean()

        # Average over the number of discriminator layers
        average_loss = total_loss / self.num_layers
        return average_loss

    def calculate_mae_loss(self, features_x, features_x_prime):
        """
        Calculate the L1 norm between two feature maps.

        Parameters
        ----------
        features_x : torch.Tensor
            Feature maps from the first set.
        features_x_prime : torch.Tensor
            Feature maps from the second set.

        Returns
        -------
        l1_distance : torch.Tensor
            L1 norm between the two sets of feature maps.
        """
        l1_distance = torch.norm(torch.abs(features_x - features_x_prime))
        return l1_distance

Main Train Script

In [ ]:
%matplotlib inline
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torch.amp import autocast, GradScaler
import os

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------
# DATASET + TRAIN/VAL SPLIT
# -------------------------
data_root = "/content/lits_png"
save_dir = "/content/drive/MyDrive/checkpoints"
os.makedirs(save_dir, exist_ok=True)

batch_size = 10

full_dataset = LiverCTDataset(data_root)

val_ratio = 0.15
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

# -------------------------
# MODELS
# -------------------------
image_channels = 1
num_discriminator_layers = 3

generator = Generator(input_channels=image_channels).to(device)
discriminator = Discriminator(input_channels=image_channels).to(device)

generator = torch.compile(generator)
discriminator = torch.compile(discriminator)

# -------------------------
# LOSS
# -------------------------
criterion = L1Loss(
    critic_network=discriminator,
    num_layers=num_discriminator_layers,
    num_training_images=batch_size
)

# -------------------------
# OPTIMIZERS
# -------------------------
generator_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# -------------------------
# MIXED PRECISION
# -------------------------
scaler = GradScaler()

# -------------------------
# VALIDATION
# -------------------------
def validate(generator, val_loader, criterion, device):
    generator.eval()
    val_loss = 0.0

    with torch.no_grad():
        for real_masks, real_images in val_loader:
            real_masks = real_masks.to(device, non_blocking=True)
            real_images = real_images.to(device, non_blocking=True)

            with autocast(device_type='cuda'):
                generated_masks = generator(real_images)
                loss = criterion(real_images, generated_masks, real_masks)

            val_loss += loss.item()

    generator.train()
    return val_loss / len(val_loader)

# -------------------------
# TRAINING LOOP
# -------------------------
num_epochs = 7
best_val_loss = float('inf')
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    epoch_g_loss = 0.0
    epoch_d_loss = 0.0

    for batch_idx, (real_masks, real_images) in enumerate(train_loader):

        real_masks = real_masks.to(device, non_blocking=True)
        real_images = real_images.to(device, non_blocking=True)

        # -----------------
        # Train Discriminator
        # -----------------
        discriminator_optimizer.zero_grad(set_to_none=True)

        with autocast(device_type='cuda'):
            fake_masks = generator(real_images)

            feat_real = discriminator.forward_all_layers(real_images * real_masks)
            feat_fake = discriminator.forward_all_layers(real_images * fake_masks.detach())

            d_loss = 0.0
            for fr, ff in zip(feat_real, feat_fake):
                d_loss += (torch.flatten(fr, 1) - torch.flatten(ff, 1)).abs().mean()
            d_loss = d_loss / num_discriminator_layers

        scaler.scale(d_loss).backward()
        scaler.step(discriminator_optimizer)
        scaler.update()

        # -----------------
        # Train Generator
        # -----------------
        for p in discriminator.parameters():
            p.requires_grad = False

        generator_optimizer.zero_grad(set_to_none=True)

        with autocast(device_type='cuda'):
            generated_masks = generator(real_images)

            # Adversarial loss from discriminator
            g_adv_loss = criterion(real_images, generated_masks, real_masks)

            # Direct pixel loss (always provides gradient)
            g_pixel_loss = (generated_masks - real_masks).abs().mean()

            # Combined: pixel loss dominates early, adversarial refines later
            g_loss = g_pixel_loss + 0.01 * g_adv_loss

        scaler.scale(g_loss).backward()
        scaler.step(generator_optimizer)
        scaler.update()

        for p in discriminator.parameters():
            p.requires_grad = True

        epoch_g_loss += g_loss.item()
        epoch_d_loss += d_loss.item()

        if batch_idx % 50 == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] "
                f"Batch [{batch_idx}/{len(train_loader)}] "
                f"D Loss: {d_loss.item():.4f} "
                f"G Loss: {g_loss.item():.4f}"
            )

    # -------------------------
    # VALIDATION
    # -------------------------
    avg_train_loss = epoch_g_loss / len(train_loader)
    avg_val_loss = validate(generator, val_loader, criterion, device)

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f"Epoch [{epoch}/{num_epochs}] Train G Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # -------------------------
    # SAVE BEST MODEL
    # -------------------------
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save({
            'epoch': epoch,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'generator_optimizer': generator_optimizer.state_dict(),
            'discriminator_optimizer': discriminator_optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
        }, os.path.join(save_dir, "best_model.pth"))
        print(f"✅ Best model saved at epoch {epoch} with Val Loss: {avg_val_loss:.4f}")

     # -------------------------
    # SAVE VALIDATION PREDICTIONS (every epoch)
    # -------------------------
    vis_dir = os.path.join(save_dir, "val_predictions", f"epoch_{epoch:04d}")
    os.makedirs(vis_dir, exist_ok=True)

    with torch.no_grad():
        sample_idx = 0
        for val_masks, val_images in val_loader:
            val_images = val_images.to(device, non_blocking=True)
            with autocast(device_type='cuda'):
                val_preds = generator(val_images)

            for i in range(val_images.size(0)):
                x = val_images[i].cpu().squeeze().numpy()
                y = val_masks[i].cpu().squeeze().numpy()
                z = val_preds[i].cpu().squeeze().numpy()

                fig, axes = plt.subplots(1, 3, figsize=(12, 4))
                axes[0].set_title("CT")
                axes[0].imshow(x, cmap="gray")
                axes[0].axis("off")
                axes[1].set_title("GT Mask")
                axes[1].imshow(y, cmap="gray")
                axes[1].axis("off")
                axes[2].set_title("Pred Mask")
                axes[2].imshow(z, cmap="gray")
                axes[2].axis("off")
                plt.tight_layout()
                plt.savefig(os.path.join(vis_dir, f"sample_{sample_idx:04d}.png"), dpi=100, bbox_inches='tight')
                plt.close('all')

                sample_idx += 1
                if sample_idx >= 100:  # only save first 100
                    break
            if sample_idx >= 100:  # only save first 100
                break

    print(f"📁 Saved {sample_idx} validation predictions to {vis_dir}")

    # Plot loss curves
    if len(train_losses) > 1:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(train_losses, label='Train Loss')
        ax.plot(val_losses, label='Val Loss')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('G Loss')
        ax.legend()
        ax.set_title('Training vs Validation Loss')
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, "loss_curve.png"), dpi=100)
        plt.show()
        plt.close('all')

  Resume Training from Checkpoint

In [ ]:
%matplotlib inline
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torch.amp import autocast, GradScaler
import os

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------
# DATASET + TRAIN/VAL SPLIT
# -------------------------
data_root = "/content/lits_png"
save_dir = "/content/drive/MyDrive/checkpoints"
os.makedirs(save_dir, exist_ok=True)

batch_size = 10

full_dataset = LiverCTDataset(data_root)

val_ratio = 0.15
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

# -------------------------
# MODELS
# -------------------------
image_channels = 1
num_discriminator_layers = 3

generator = Generator(input_channels=image_channels).to(device)
discriminator = Discriminator(input_channels=image_channels).to(device)

generator = torch.compile(generator)
discriminator = torch.compile(discriminator)

# -------------------------
# LOSS
# -------------------------
criterion = L1Loss(
    critic_network=discriminator,
    num_layers=num_discriminator_layers,
    num_training_images=batch_size
)

# -------------------------
# OPTIMIZERS
# -------------------------
generator_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# -------------------------
# MIXED PRECISION
# -------------------------
scaler = GradScaler()

# -------------------------
# LOAD CHECKPOINT
# -------------------------
checkpoint = torch.load(os.path.join(save_dir, "best_model.pth"), map_location=device)

generator.load_state_dict(checkpoint['generator_state_dict'])
discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
generator_optimizer.load_state_dict(checkpoint['generator_optimizer'])
discriminator_optimizer.load_state_dict(checkpoint['discriminator_optimizer'])

start_epoch = checkpoint['epoch'] + 1
best_val_loss = checkpoint['val_loss']
train_losses = []
val_losses = []

print(f"✅ Loaded checkpoint from epoch {checkpoint['epoch']} with Val Loss: {checkpoint['val_loss']:.4f}")

# -------------------------
# VALIDATION
# -------------------------
def validate(generator, val_loader, criterion, device):
    generator.eval()
    val_loss = 0.0

    with torch.no_grad():
        for real_masks, real_images in val_loader:
            real_masks = real_masks.to(device, non_blocking=True)
            real_images = real_images.to(device, non_blocking=True)

            with autocast(device_type='cuda'):
                generated_masks = generator(real_images)
                loss = criterion(real_images, generated_masks, real_masks)

            val_loss += loss.item()

    generator.train()
    return val_loss / len(val_loader)

# -------------------------
# RESUME TRAINING
# -------------------------
num_epochs = 7

for epoch in range(start_epoch, num_epochs):
    epoch_g_loss = 0.0
    epoch_d_loss = 0.0

    for batch_idx, (real_masks, real_images) in enumerate(train_loader):

        real_masks = real_masks.to(device, non_blocking=True)
        real_images = real_images.to(device, non_blocking=True)

        # -----------------
        # Train Discriminator
        # -----------------
        discriminator_optimizer.zero_grad(set_to_none=True)

        with autocast(device_type='cuda'):
            fake_masks = generator(real_images)

            feat_real = discriminator.forward_all_layers(real_images * real_masks)
            feat_fake = discriminator.forward_all_layers(real_images * fake_masks.detach())

            d_loss = 0.0
            for fr, ff in zip(feat_real, feat_fake):
                d_loss += (torch.flatten(fr, 1) - torch.flatten(ff, 1)).abs().mean()
            d_loss = d_loss / num_discriminator_layers

        scaler.scale(d_loss).backward()
        scaler.step(discriminator_optimizer)
        scaler.update()

        # -----------------
        # Train Generator
        # -----------------
        for p in discriminator.parameters():
            p.requires_grad = False

        generator_optimizer.zero_grad(set_to_none=True)

        with autocast(device_type='cuda'):
            generated_masks = generator(real_images)
            g_adv_loss = criterion(real_images, generated_masks, real_masks)
            g_pixel_loss = (generated_masks - real_masks).abs().mean()
            g_loss = g_pixel_loss + 0.01 * g_adv_loss

        scaler.scale(g_loss).backward()
        scaler.step(generator_optimizer)
        scaler.update()

        for p in discriminator.parameters():
            p.requires_grad = True

        epoch_g_loss += g_loss.item()
        epoch_d_loss += d_loss.item()

        if batch_idx % 50 == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] "
                f"Batch [{batch_idx}/{len(train_loader)}] "
                f"D Loss: {d_loss.item():.4f} "
                f"G Loss: {g_loss.item():.4f}"
            )

    # -------------------------
    # VALIDATION
    # -------------------------
    avg_train_loss = epoch_g_loss / len(train_loader)
    avg_val_loss = validate(generator, val_loader, criterion, device)

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f"Epoch [{epoch}/{num_epochs}] Train G Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # -------------------------
    # SAVE BEST MODEL
    # -------------------------
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save({
            'epoch': epoch,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'generator_optimizer': generator_optimizer.state_dict(),
            'discriminator_optimizer': discriminator_optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
        }, os.path.join(save_dir, "best_model.pth"))
        print(f"✅ Best model saved at epoch {epoch} with Val Loss: {avg_val_loss:.4f}")

    # -------------------------
    # SAVE VALIDATION PREDICTIONS
    # -------------------------
    vis_dir = os.path.join(save_dir, "val_predictions", f"epoch_{epoch:04d}")
    os.makedirs(vis_dir, exist_ok=True)

    with torch.no_grad():
        sample_idx = 0
        for val_masks, val_images in val_loader:
            val_images = val_images.to(device, non_blocking=True)
            with autocast(device_type='cuda'):
                val_preds = generator(val_images)

            for i in range(val_images.size(0)):
                x = val_images[i].cpu().squeeze().numpy()
                y = val_masks[i].cpu().squeeze().numpy()
                z = val_preds[i].cpu().squeeze().numpy()

                fig, axes = plt.subplots(1, 3, figsize=(12, 4))
                axes[0].set_title("CT")
                axes[0].imshow(x, cmap="gray")
                axes[0].axis("off")
                axes[1].set_title("GT Mask")
                axes[1].imshow(y, cmap="gray")
                axes[1].axis("off")
                axes[2].set_title("Pred Mask")
                axes[2].imshow(z, cmap="gray")
                axes[2].axis("off")
                plt.tight_layout()
                plt.savefig(os.path.join(vis_dir, f"sample_{sample_idx:04d}.png"), dpi=100, bbox_inches='tight')
                plt.close('all')

                sample_idx += 1
                if sample_idx >= 100:
                    break
            if sample_idx >= 100:
                break

    print(f"📁 Saved {sample_idx} validation predictions to {vis_dir}")

    if len(train_losses) > 1:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(train_losses, label='Train Loss')
        ax.plot(val_losses, label='Val Loss')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('G Loss')
        ax.legend()
        ax.set_title('Training vs Validation Loss')
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, "loss_curve.png"), dpi=100)
        plt.show()
        plt.close('all')